In [1]:
import pandas as pd         
import matplotlib.pyplot as plt
from utilities import  create_bank_col, clean_tweet
#Reading data and models
import glob               
import os
import re
import numpy as np
import datetime as dt

## Load in Twitter dataset

In [2]:
# Read the path
dir_datasets = os.getenv("DATASETS_PATH")
directory = f"{dir_datasets}/tweets_of_the_top_5_banks_in_SA/"

# Use os.listdir() to get a list of all files in the directory
files = os.listdir(directory)

# Use a list comprehension to filter out only CSV files
csv_files = [file for file in files if file.endswith(".csv")]

# Initialize an empty list to store the dataframes
df_list = []

# Iterate over the CSV files and read them into pandas dataframes
for file in csv_files:
    file_path = os.path.join(directory, file)
    df = pd.read_csv(file_path)
    df_list.append(df)

# Concatenate the dataframes
tweets_df = pd.concat(df_list)

/tmp/ipykernel_20313/1788197953.py:17: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [5]:
tweets_df = pd.read_csv("twitter_data_2019-01-01_to_2019-01-02.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'twitter_data_2019-01-01_to_2019-01-02.csv'

## Exploratory Data Analysis

In [3]:
# Top records of dataset
tweets_df.head()

,Unnamed: 0,id,conversation_id,created_at,date,timezone,place,base_tweet,cleaned_tweet,language,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,0,1.358170e+18,1.358170e+18,1.612650e+12,2021/02/06 21:42,0,NaN,@magefalco Let me know when you would like to ...,let me know when you would like to play agains...,en,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'magefalco', 'name': 'PHNM | ...",NaN,NaN,NaN,NaN
1,1,1.358170e+18,1.358030e+18,1.612650e+12,2021/02/06 21:40,0,NaN,@Thabiso_TK Happy birthday to the handsome msu...,happy birthday to the handsome msushwana may h...,en,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'Thabiso_TK', 'name': 'Thabis...",NaN,NaN,NaN,NaN
2,2,1.358170e+18,1.358170e+18,1.612650e+12,2021/02/06 21:40,0,NaN,@SbuMasang JZ will give us Nedbank Cup. And n...,jz will give us nedbank cup and next season t...,en,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'SbuMasang', 'name': 'Masango...",NaN,NaN,NaN,NaN
3,3,1.358170e+18,1.358170e+18,1.612650e+12,2021/02/06 21:31,0,NaN,This is result of SA Nedbank cup ———————————...,this is result of sa nedbank cup february ...,en,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,4,1.358160e+18,1.358090e+18,1.612650e+12,2021/02/06 21:14,0,NaN,@Radebe_merci Invest bank &amp; Absa,invest bank amp absa,en,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'Radebe_merci', 'name': '🎀Str...",NaN,NaN,NaN,NaN


In [4]:
# Columns and features in data
tweets_df.columns

Index(['Unnamed: 0', 'id', 'conversation_id', 'created_at', 'date', 'timezone',
       'place', 'base_tweet', 'cleaned_tweet', 'language', 'hashtags',
       'cashtags', 'user_id', 'user_id_str', 'username', 'name', 'day', 'hour',
       'link', 'urls', 'photos', 'video', 'thumbnail', 'retweet', 'nlikes',
       'nreplies', 'nretweets', 'quote_url', 'search', 'near', 'geo', 'source',
       'user_rt_id', 'user_rt', 'retweet_id', 'reply_to', 'retweet_date',
       'translate', 'trans_src', 'trans_dest'],
      dtype='object')

In [5]:
# Length of dataset
print('Length of data is', len(tweets_df))

Length of data is 1085729


In [6]:
# Dataset information
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1085729 entries, 0 to 376539
Data columns (total 40 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   Unnamed: 0       1085729 non-null  int64  
 1   id               1085729 non-null  float64
 2   conversation_id  1085729 non-null  float64
 3   created_at       1085729 non-null  float64
 4   date             1085729 non-null  object 
 5   timezone         1085729 non-null  int64  
 6   place            964 non-null      object 
 7   base_tweet       1085729 non-null  object 
 8   cleaned_tweet    1001735 non-null  object 
 9   language         1085729 non-null  object 
 10  hashtags         1085729 non-null  object 
 11  cashtags         1085729 non-null  object 
 12  user_id          1085729 non-null  float64
 13  user_id_str      1085729 non-null  float64
 14  username         1085729 non-null  object 
 15  name             1085687 non-null  object 
 16  day              10

In [7]:
# Checking for Null values
print(f'Number of Null values: {np.sum(df.isnull().any(axis=1))}')

Number of Null values: 376540


In [8]:
# cleaned_tweets, hashtags = clean_tweet(tweets_df['base_tweet'])
# # Add the cleaned tweets and hashtags columns to the dataframe
# tweets_df['clean_tweet'] = cleaned_tweets
# tweets_df['hashtags'] = hashtags

In [9]:
import re
import numpy as np
import emoji

def translate_emoji(tweet):
    if tweet == None or tweet == "":
      tweet = tweet
    else:
      tweet = emoji.demojize(tweet).replace(":", "").replace("_", " ")
    return tweet

def remove_email(tweet):
    email = re.compile(r'[\w\.-]+@[\w\.-]+')
    return email.sub(r'',tweet)
     
def clean_tweet(tweets):
    cleaned_tweets = []
    hashtags = []
    for tweet in tweets:
        
        # Remove mentions
        tweet = re.sub(r'@\w+', '', tweet)
        
        # Remove links
        tweet = re.sub(r'http\S+', '', tweet)
        tweet = re.sub(r'www\S+', '', tweet)
        tweet = re.sub(r'bit.ly/\S+', '', tweet) # remove bitly links
        tweet = tweet.strip('[link]') # remove [links]
        
        # Remove email address
        tweet = remove_email(tweet)
        
        # Translate emojis
        tweet = translate_emoji(tweet)
        
        # Capture hashtags
        hashtag_list = re.findall(r'#\w+', tweet)
        if len(hashtag_list) == 0:
            hashtag_list = []
            
        hashtags.append(hashtag_list[1:])
        
        # Remove hashtags
        tweet = re.sub(r"#(\w+)", '', tweet)
        
        # Remove &amp
        tweet = re.sub(r'&amp ', '', tweet)
        
        # Remove special characters
        tweet = re.sub('([_]+)', "", tweet)
        
        # remove any unnecessary spaces
        tweet = " ".join(tweet.split())
        
        tweet = "".join(i for i in tweet if ord(i)<128)
        
        cleaned_tweets.append(tweet)
    return cleaned_tweets, hashtags

cleaned_tweets, hashtags = clean_tweet(tweets_df['base_tweet'].head(5))
# Add the cleaned tweets and hashtags columns to the dataframe
tweets_df['clean_tweet'] = cleaned_tweets
tweets_df['hashtags'] = hashtags

ValueError: Length of values (5) does not match length of index (1085729)

In [ ]:
tweets_df[['base_tweet', 'cleaned_tweet', 'clean_tweet']].head()